In [ ]:
reference_data = [
  {
    "question": "What is the company's policy on remote work?", 
    "ground_truth": "Remote work is allowed up to 3 days per week. ", #Expected llm generated answer
    "context": "Remote work is allowed up to 3 days per week." #Expected retrieved context
  }
]
question = reference_data[0]['question']
ground_truth = reference_data[0]['ground_truth']
context = reference_data[0]['context']
print (f"question: {question}")
print (f"ground_truth: {ground_truth}")
print (f"context: {context}")

In [ ]:
# Retrieve context from Milvus DB

from milvus_chatbot_with_rag import retrieve_similiar_contexts

def perform_retrieval(question):

    retrieved_context = retrieve_similiar_contexts(question, "employee_policies", 1)[0]['content']
    print (f"perform_retrieval.retrieved_context: {retrieved_context}")
    return retrieved_context

# Validate retrieved context against reference data    
perform_retrieval("What is the company's policy on remote work?")

In [1]:
%pip install ragas datasets 

   ---------------------------------------- 0.0/27.6 MB ? eta -:--:--
   ------ --------------------------------- 4.2/27.6 MB 20.2 MB/s eta 0:00:02
   ------------ --------------------------- 8.7/27.6 MB 21.0 MB/s eta 0:00:01
   ------------------ --------------------- 12.8/27.6 MB 21.3 MB/s eta 0:00:01
   ----------------------- ---------------- 16.5/27.6 MB 20.1 MB/s eta 0:00:01
   ------------------------------- -------- 21.8/27.6 MB 21.2 MB/s eta 0:00:01
   ------------------------------------ --- 24.9/27.6 MB 20.3 MB/s eta 0:00:01
   ---------------------------------------  27.5/27.6 MB 20.3 MB/s eta 0:00:01
   ---------------------------------------- 27.6/27.6 MB 19.2 MB/s  0:00:01
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ------------------ --------------------- 4.7/10.4 MB 24.4 MB/s eta 0:00:01
   ---------------------------------------  10.2/10.4 MB 25.7 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 24.1 MB/s  0:00:00


In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall

from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

# question = reference_data[0]['question']
# ground_truth = reference_data[0]['ground_truth']
# context = reference_data[0]['context']
# Question User asked
question = reference_data[0]['question']

# Reference context (should be a string)
reference_context = reference_data[0]['context']

# Retrieved context (a string from perform_retrieval)
retrieved_context = [perform_retrieval(question)]

# Build dataset properly
dataset_dict = {
    "question": [question],
    "contexts": [retrieved_context],      # list of strings INSIDE another list
    "ground_truth": [reference_context],   # single string
    "answer": [""]
}

print(f"dataset_dict: {dataset_dict}")

ragas_dataset = Dataset.from_dict(dataset_dict)

# Evaluate retrieval
results = evaluate(
    dataset=ragas_dataset,
    metrics=[context_precision, context_recall]
)

In [ ]:
print("Retrieval Evaluation Results:")
results.to_pandas()